In [3]:
import os
from dotenv import load_dotenv
from openai import OpenAI

In [4]:
import gradio as gr

In [5]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
openai=OpenAI(api_key=openai_api_key)

In [6]:
from litellm import completion

In [7]:
system_message = "You are a helpful assistant"
def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
     ]
    response = completion(
        model="openai/gpt-5-nano",
        messages=messages
    )
    return response.choices[0].message.content


In [8]:
message_gpt("What is today's date?")

"Today's date is December 25, 2025 (2025-12-25)."

In [9]:
def shout(text):
    print(f'Shout has been called with the input {text}')
    return text.upper()

In [10]:
shout("hello world")

Shout has been called with the input hello world


'HELLO WORLD'

In [11]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


### Adding user id and password

In [12]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True, auth=("hassan", "muhammad"))

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [13]:
message_input = gr.Textbox(label="Your message:", info="Enter a message to be shouted", lines=7)
message_output = gr.Textbox(label="Response:", lines=8)

view = gr.Interface(
    fn=shout,
    title="Shout", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=["hello", "howdy"], 
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [14]:
message_input = gr.Textbox(label="Your message:", info="Enter a message for GPT-5 Nano", lines=7)
message_output = gr.Textbox(label="Response:", lines=8)

view = gr.Interface(
    fn=message_gpt,
    title="GPT", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=["hello", "howdy"], 
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


In [15]:
system_message = "You are a helpful assistant that responds in markdown without code blocks"

message_input = gr.Textbox(label="Your message:", info="Enter a message for GPT-5-nano", lines=7)
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=message_gpt,
    title="GPT", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI engineer",
        ], 
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


In [16]:
def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
     ]
    response = completion(
        model="openai/gpt-5-nano",
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in response:
        result += chunk.choices[0].delta.content or ''
        yield result


In [17]:
message_input = gr.Textbox(label="Your message:", info="Enter a message for GPT-5-nano", lines=7)
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_gpt,
    title="GPT", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI engineer",
        ], 
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


In [18]:
def stream_gemma(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = completion(
        model='ollama/gemma3:4b',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [19]:
message_input = gr.Textbox(label="Your message:", info="Enter a message for Gemma 3:4b", lines=7)
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_gemma,
    title="Gemma", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI engineer",
        ], 
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


In [20]:
def stream_model(prompt, model):
    if model == "GPT":
        result = stream_gpt(prompt)
    elif model == "Gemma":
        result = stream_gemma(prompt)
    else:
        raise ValueError("Unsupported model")
    yield from result

In [21]:
input_message = gr.Textbox(label="Your message:", info="Enter a message for the selected model", lines=7)
select_model = gr.Dropdown(choices=["GPT", "Gemma"], label="Select Model", value="GPT")
output_response = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_model,
    inputs=[input_message, select_model],
    outputs=[output_response],
    examples=[
        ["Explain the Transformer architecture to a layperson", "GPT"],
        ["Explain the Transformer architecture to an aspiring AI engineer", "Gemma"],
    ],
    title="Multi-Model Streamer",
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.


In [25]:
system_message = """
You are an assistant that analyzes the contents of a company website landing page
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
"""
from scraper import fetch_website_contents

In [26]:
def stream_brochure(company_name, url, model):
    yield ''
    prompt = f"""Create a brochure for the company {company_name} based on the information available on their website: {url}."""
    prompt += f"\n\nWebsite Contents:\n{fetch_website_contents(url)}"
    if model == "GPT":
        result = stream_gpt(prompt)
    elif model == "Gemma":
        result = stream_gemma(prompt)
    else:
        raise ValueError("Unsupported model")
    yield from result

In [28]:
company_name_input = gr.Textbox(label="Company Name:", info="Enter the name of the company", lines=1)
website_url_input = gr.Textbox(label="Website URL:", info="Enter the company's website URL", lines=1)
select_model = gr.Dropdown(choices=["GPT", "Gemma"], label="Select Model", value="GPT")
output_brochure = gr.Markdown(label="Brochure:")

view = gr.Interface(
    fn=stream_brochure,
    inputs=[company_name_input, website_url_input, select_model],
    outputs=[output_brochure],
    title="Company Brochure Generator",
    examples=[
            ["Hugging Face", "https://huggingface.co", "GPT"],
            ["Edward Donner", "https://edwarddonner.com", "Gemma"],
        ], 
    flagging_mode="never"
    )
view.launch(share=True)

* Running on local URL:  http://127.0.0.1:7870
* Running on public URL: https://b1ead4a45c25c24cf7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
